In [ ]:
from copy import deepcopy
limit = 10*1024
overLimit = lambda off: calcSize(off) > limit
calcSize = lambda dct: len(json.dumps(dct))

def strippedResponse(response):
  stripped = {'score': response['score']}
  if 'percentiles' in response and 'similar' in response['percentiles']:
    stripped['percentiles'] = {'similar': response['percentiles']['similar']}
  return stripped

def stripReport(reportOrProf):
  if 'responses' in reportOrProf:
    for attribute, response in reportOrProf['responses'].iteritems():
      reportOrProf['responses'][attribute] = strippedResponse(response)
  return reportOrProf

def trimmedReports(reports, targetSize):
  trimmed = deepcopy(reports)
  for i, report in enumerate(reversed(trimmed)):
    stripReport(report)
    if 'profs' in report:
      report['profs'] = [stripReport(p) for p in report['profs']]        
    if calcSize(trimmed) < targetSize:
#       print '  Got under size limit by stripping', i + 1, 'of', len(reports)
      return trimmed 
  while calcSize(trimmed) > targetSize:
#     print '  Still too big', calcSize(trimmed)
    if len(trimmed) > 1:
      trimmed = trimmed[:-1]
    else:
#       print '  Removed all', len(reports), 'to get under size limit.'
      return []
#   print '  Removed', len(reports), '->', len(trimmed), 'to get under size limit.',
#   print '  Last:', reports[-1]['term'], reports[-1]['year'], '->', trimmed[-1]['term'], trimmed[-1]['year']
  return trimmed

def trimProfs(newOffering):
  for prof in newOffering['profs']:
    if 'matched' in prof:
      del prof['matched']

def trimTopReport(newOffering):
  if 'topReport' in newOffering:
    top = newOffering['topReport']
    del newOffering['topReport']
    trimmedTop = trimmedReports([top], 10*1024 - calcSize(newOffering))
    if len(trimmedTop) > 0:
      newOffering['topReport'] = trimmedTop[0]
      
def trimReports(newOffering):
  oldReports = [r for r in newOffering['reports'] if r['year'] < '2014']
  newReports = [r for r in newOffering['reports'] if r['year'] >= '2014']
  del newOffering['reports']
  if overLimit(newOffering):
    trimTopReport(newOffering)
  if (calcSize(newOffering) + calcSize(newReports)) < limit:
    newOffering['reports'] = newReports + trimmedReports(oldReports, limit - (calcSize(newOffering) + calcSize(newReports))) 
  elif calcSize(newOffering) < limit:
    trimmedNew = trimmedReports(newReports, limit - calcSize(newOffering))
    if len(trimmedNew) > 0:
      newOffering['reports'] = trimmedNew

from collections import Counter
def algoliaCompress(offerings):
  algoliaOfferings = []
  for offering in offerings:
    if overLimit(offering):
      newOffering = deepcopy(offering)
      if 'reports' in newOffering:
        trimReports(newOffering)
      if overLimit(newOffering):
        trimTopReport(newOffering)
      if overLimit(newOffering):
        trimProfs(newOffering)
      if overLimit(newOffering):
        raise
      else:
        print offering['group'], offering['number']
        print '  from', calcSize(offering), '->', calcSize(newOffering)
        algoliaOfferings.append(newOffering)
    else:
      algoliaOfferings.append(offering)
  return algoliaOfferings


In [ ]:
algoliaOfferings = algoliaCompress(offerings)

In [ ]:
from algoliasearch import algoliasearch
client = algoliasearch.Client("D86FK05FUD", '0cebfab0abe857cd36dd70e37f043a64')
index = client.init_index("offerings")
index.save_objects(algoliaOfferings)